In [113]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [156]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [188]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['date'] = pd.to_datetime(df['date']).dt.strftime("%Y-%m-%d")
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

### Settlements data

In [283]:
settlements_data = pd.DataFrame(workbook.worksheet('settlements').get_all_records()).set_index('date').reset_index()

settlements_data.groupby(['paid_by', 'paid_to']).sum('amount')

,,amount
paid_by,paid_to,
prateek,raghotham,368.33


### Datewise expenditure

In [158]:
expense_data = pd.DataFrame(workbook.worksheet('expense_tracker').get_all_records()).set_index('date').reset_index()

In [160]:
expense_data.head()

,date,amount,paid_by
0,2023-03-05,380,ajay
1,2023-03-06,200,raghotham
2,2023-03-07,200,ajay
3,2023-03-08,270,raghotham
4,2023-01-03,500,raghotham


### players attended on date

In [161]:
def fn(x):
    return [i for i in set(', '.join([', '.join(x[i]) for i in range(4)]).split(', ')) if i != 'other']

players_on_date = df[['date', 'team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].copy()

players_on_date['date'] = players_on_date['date'].apply(lambda x: f'{datetime.strptime(x, "%m/%d/%Y").date()}')

players_on_date = players_on_date.groupby("date").agg({
    'team_1_player_1': 'unique',
    'team_1_player_2': 'unique',
    'team_2_player_1': 'unique',
    'team_2_player_2': 'unique'
})

players_on_date['players'] = players_on_date[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].apply(fn, axis=1)
players_on_date = players_on_date[['players']]
players_on_date['number_of_players'] = players_on_date['players'].str.len()

In [163]:
players_on_date.sort_values('date').tail()

,players,number_of_players
date,,
2023-03-02,"[ajay, vinay, aakarsh, raghotham, prateek]",5
2023-03-05,"[ajay, vinay, raghotham, srinidhi, prateek]",5
2023-03-06,"[ajay, vinay, raghotham, sameer]",4
2023-03-07,"[ajay, vinay, raghotham, sameer, prateek]",5
2023-03-08,"[ajay, vinay, aakarsh, raghotham, sameer, srin...",6


### daily player share

In [164]:
player_share = pd.merge(players_on_date.explode('players'), expense_data, how="inner", on="date")
player_share['share'] = player_share['amount'] / player_share['number_of_players']

In [165]:
player_share.tail()

,date,players,number_of_players,amount,paid_by,share
114,2023-03-08,vinay,6,270,raghotham,45.0
115,2023-03-08,aakarsh,6,270,raghotham,45.0
116,2023-03-08,raghotham,6,270,raghotham,45.0
117,2023-03-08,sameer,6,270,raghotham,45.0
118,2023-03-08,srinidhi,6,270,raghotham,45.0


### total player balances

In [167]:
# balances = 
player_share.groupby('players').agg({
    'date': 'count',
    'share': 'sum'
}).round(decimals=2)

,date,share
players,,
aakarsh,13,561.67
ajay,23,1061.83
nithin,4,163.33
prateek,10,484.33
raghotham,23,1061.83
ranga,1,50.00
sameer,13,580.00
srinidhi,11,535.17
vinay,21,1001.83


### player balances to others

In [168]:
balances = player_share.groupby(['players', 'paid_by'][::-1]).agg({
    'date': 'count',
    'share': 'sum'
}).reset_index()

balances = balances[balances['paid_by'] != balances['players']]

In [169]:
balances = pd.merge(
    balances,
    balances,
    left_on=['paid_by', 'players'],
    right_on=['players', 'paid_by'],
    how="left"
)
# balances

In [170]:
balances['owes'] = np.where(
    balances['paid_by_y'].isna(),
    balances['share_x'],
    np.where(
        balances['share_x'] > balances['share_y'],
        balances['share_x'] - balances['share_y'],
        0
    )
)

In [171]:
balances = balances[['players_x', 'paid_by_x', 'owes', 'date_x']]

balances.columns = ['player', 'owes_to', 'amount', 'for_days']

In [172]:
balances

,player,owes_to,amount,for_days
0,aakarsh,ajay,50.000000,1
1,prateek,ajay,116.000000,2
2,raghotham,ajay,0.000000,3
3,sameer,ajay,90.000000,2
4,srinidhi,ajay,76.000000,1
5,vinay,ajay,166.000000,3
6,aakarsh,raghotham,511.666667,12
7,ajay,raghotham,729.833333,20
8,nithin,raghotham,163.333333,4
9,prateek,raghotham,368.333333,8


#### Post settlements

In [178]:
balances_post_settlement = pd.merge(
    balances,
    settlements_data,
    left_on=['player', 'owes_to'],
    right_on=['paid_by', 'paid_to'],
    how = "left"
)

balances_post_settlement['amount'] = np.where(
    balances_post_settlement['paid_to'].isna(),
    balances_post_settlement['amount_x'],
    np.where(
        balances_post_settlement['amount_x'] > balances_post_settlement['amount_y'],
        balances_post_settlement['amount_x'] - balances_post_settlement['amount_y'],
        0
    )
)

balances_post_settlement = balances_post_settlement[['player', 'owes_to', 'amount']].round(decimals=2)

In [179]:
balances_post_settlement

,player,owes_to,amount
0,aakarsh,ajay,50.00
1,prateek,ajay,116.00
2,raghotham,ajay,0.00
3,sameer,ajay,90.00
4,srinidhi,ajay,76.00
5,vinay,ajay,166.00
6,aakarsh,raghotham,511.67
7,ajay,raghotham,729.83
8,nithin,raghotham,163.33
9,prateek,raghotham,0.00


### Venue wise expenditure

In [192]:
venue_wise_expenditure = pd.merge(
    expense_data,
    df[['date', 'venue']].drop_duplicates(),
    left_on='date',
    right_on='date'
).groupby('venue').sum('amount').reset_index()

In [194]:
venue_wise_expenditure

,venue,amount
0,infinity badminton academy,500
1,isro match point,950
2,match point - gublaala,3550
3,ssba - nps,500


In [282]:
px.pie(
    venue_wise_expenditure,
    values="amount",
    names='venue',
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.OrRd_r[-venue_wise_expenditure.shape[0]:],
    hole=0.3
)

### Monthy expenditure

In [239]:
monthly_expenditure = expense_data.copy()
monthly_expenditure['month'] = pd.to_datetime(monthly_expenditure['date']).dt.month

monthly_expenditure = monthly_expenditure.groupby(
    pd.to_datetime(monthly_expenditure['date']).dt.month_name()
).agg({'amount': 'sum', 'month': 'min'})

monthly_expenditure['color'] = np.where(
    monthly_expenditure['amount'] == monthly_expenditure['amount'].max(),
    '#b5de2b',
    'lightslategrey'
)

monthly_expenditure = monthly_expenditure.sort_values('month', ascending=False)

monthly_expenditure

,amount,month,color
date,,,
March,1250,3,lightslategrey
February,1900,2,lightslategrey
January,2350,1,#b5de2b


In [240]:
go.Figure(
    go.Bar(
        y=monthly_expenditure.index,
        x=monthly_expenditure['amount'],
        orientation='h',
        marker_color=monthly_expenditure['color'],
#         hovertemplate="Win Percentage: %{x} %"
    )
)